# Emotions Recognition ai project

### 사용할 모델 다운로드
#### face-detection-adas-0001 다운로드

In [2]:
! omz_downloader --name face-detection-adas-0001 --precision FP16

################|| Downloading face-detection-adas-0001 ||################

========== Downloading c:\BrainAI1\emotions-recognition\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.xml
... 100%, 304 KB, 330 KB/s, 0 seconds passed

========== Downloading c:\BrainAI1\emotions-recognition\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.bin
... 49%, 1024 KB, 172 KB/s, 5 seconds passed
... 99%, 2048 KB, 280 KB/s, 7 seconds passed
... 100%, 2056 KB, 281 KB/s, 7 seconds passed



In [3]:
! omz_downloader --name emotions-recognition-retail-0003 --precision FP16

################|| Downloading emotions-recognition-retail-0003 ||################

========== Downloading c:\BrainAI1\emotions-recognition\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.xml
... 100%, 54 KB, 47 KB/s, 1 seconds passed

========== Downloading c:\BrainAI1\emotions-recognition\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.bin
... 21%, 1024 KB, 744 KB/s, 1 seconds passed
... 42%, 2048 KB, 1272 KB/s, 1 seconds passed
... 63%, 3072 KB, 1873 KB/s, 1 seconds passed
... 84%, 4096 KB, 2222 KB/s, 1 seconds passed
... 100%, 4848 KB, 2608 KB/s, 1 seconds passed



In [4]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

### INFERENCE 할 DEVICE 확인

In [6]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

### Load the model

In [78]:
core = ov.Core()

model = core.read_model(model='./models/face-detection-adas-0001.xml')
emotion_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


In [79]:
model = core.read_model(model='./models/emotions-recognition-retail-0003.xml')
emotion_model = core.compile_model(model=model, device_name="CPU")

emotion_input_layer = emotion_model.input(0)
emotion_output_layer = emotion_model.output(0)

print("Input layer shape: ", emotion_input_layer.shape)
print("Output layer shape:", emotion_output_layer.shape)

Input layer shape:  [1,3,64,64]
Output layer shape: [1,5,1,1]


### Load image

In [80]:
frame = cv2.imread("image/emotions.jpg")

resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

### DrawBouningBoxes() 함수만들기

In [81]:
def DrawBoundingBoxes(output, image, conf=0.5):
    boxes=[]
    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
        boxes.apped(box)
   
    return canvas

In [82]:
def DrawBoundingBoxes(output, frame, conf=0.5):
    boxes = []
    canvas = frame.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = (detection[3:7] * np.array([w, h, w, h])).astype("int") # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box   # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 그리기
        boxes.append(box)     #이미지에 박스를 그린 얼굴의 위치 저장
   
    return boxes

In [83]:
def DrawText(output, frame, face_position):
    # emotions 딕셔너리 생성
    emotions = {
        0:"neutral",
        1:"happy",
        2:"sad",
        3:"surprise",
        4:"anger"
    }
    # 딕셔너리 출력하기
    #for key, value in emotions.items():
    #    print(key, value, end='      ')
    #print()
        
    predictions = output[0,:,0,0]              # 5개의 감정 예측값 저장
    print("predictions : " + str(predictions))
    
    topresult_index = np.argmax(predictions)   # 5개의 감정 예측값 중 가장 높은값의 위치 저장
    #print("topresult_index : " + str(topresult_index))
    
    emotion = emotions[topresult_index]        # emotions에서 topresult_index 값에 해당하는 감정 저장
    #print("emotion : " + emotion)
    
    cv2.putText(frame, emotion,                 # 예측한 감정값 이미지에 출력하기
                (face_position[0],face_position[1]),    #xmin, ymin 값을 가져와 위치 설정
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (255, 0,0), 2)

In [84]:
face_output = face_model([input_frame])[face_output_layer]
boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)

In [85]:
if boxes is not None:
    
    for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
        xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
        emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
        # 감정 인식 모델을 사용하기 위해 이미지 전처리
        # Input layer shape:  [1,3,64,64]
        resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
        transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
        input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

        emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
        DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기

predictions : [0.472979   0.00922723 0.03208937 0.12683505 0.35886937]
predictions : [1.8001714e-04 1.5403769e-03 9.2567647e-01 2.4386555e-04 7.2359294e-02]
predictions : [5.2245511e-03 7.2852528e-04 8.5551362e-04 9.9174678e-01 1.4446168e-03]
predictions : [1.6682513e-01 8.2122630e-01 9.5314812e-03 1.8027073e-04 2.2369029e-03]
predictions : [3.9358638e-04 9.7996461e-01 1.5637850e-02 2.5980277e-03 1.4059981e-03]
predictions : [5.8812648e-04 5.6197602e-02 1.8172228e-04 9.1954255e-01 2.3489922e-02]
predictions : [6.7086075e-05 9.9930334e-01 7.4545591e-05 4.5859793e-04 9.6366646e-05]
predictions : [5.3750403e-04 1.4339936e-04 1.6935866e-02 4.8260146e-05 9.8233497e-01]
predictions : [0.0036957  0.9840232  0.00687442 0.00285215 0.00255446]
predictions : [1.0526952e-01 3.5744105e-04 7.9664819e-02 1.4745106e-04 8.1456077e-01]
predictions : [4.2153178e-03 8.6598091e-02 1.0233697e-04 8.9234114e-01 1.6743194e-02]
predictions : [1.6983333e-03 5.4251018e-04 3.2775700e-03 1.5957872e-05 9.9446565e-01

In [86]:
cv2.imshow("emotion-recognition", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [87]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 400
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [88]:
background = "./image/Background2.jpg"  #사용할 배경화면 경로

bg = cv2.imread(background)
deployment = AddBackground(frame, bg)

In [89]:
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [91]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./image/Background2.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]
    
    boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)
    
    if boxes is not None:
    
        for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
            xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
            emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
            # 감정 인식 모델을 사용하기 위해 이미지 전처리
            # Input layer shape:  [1,3,64,64]
            resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
            transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
            input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

            emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
            DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기
    
    deployment = AddBackground(frame, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()

predictions : [0.29041976 0.0309211  0.10452626 0.00525108 0.56888175]
predictions : [0.2558909  0.02146551 0.11981048 0.00391119 0.5989219 ]
predictions : [0.17208959 0.01741841 0.15480074 0.00269453 0.6529967 ]
predictions : [0.24217407 0.03205158 0.1382364  0.00450833 0.58302957]
predictions : [0.24145117 0.0279613  0.09974975 0.00268331 0.62815446]
predictions : [0.17862847 0.01084372 0.09252564 0.00293697 0.71506506]
predictions : [0.18233785 0.01571486 0.15320411 0.003042   0.6457013 ]
predictions : [0.23657809 0.00833921 0.1131467  0.00922585 0.6327102 ]
predictions : [0.2118876  0.00425938 0.10558891 0.0107461  0.667518  ]
predictions : [0.24164732 0.00649723 0.0982643  0.01661449 0.63697666]
predictions : [0.32886675 0.01196046 0.12287752 0.02186635 0.5144289 ]
predictions : [0.3703492  0.01583659 0.12861824 0.02313884 0.46205705]
predictions : [0.35569796 0.00909118 0.1500311  0.03014392 0.4550358 ]
predictions : [0.2614041  0.01094993 0.1332401  0.01685794 0.5775479 ]
predic